In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
print(sys.path)

import Models.autoencoders as autoencoders
import Double_Pendulum.Lumped_Mass.robot_parameters as robot_parameters
import Double_Pendulum.Lumped_Mass.transforms as transforms
import Double_Pendulum.Lumped_Mass.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS
plotter = pendulum_plot.Anim_plotter(rp)
neural_net = False

if neural_net:
    model = autoencoders.Autoencoder_double(rp).to(device)
    model_location = '../Models/Split_AEs/ana_theta_loss.pth'
    model.load_state_dict(torch.load(model_location, weights_only=True))
else:
    model = autoencoders.Analytic_transformer(rp)

In [ ]:
def check_clockwise(q):
    if (q[0,1] >= q[0,0] and q[0,1] <= q[0,0] + torch.pi) or (q[0,1] >= q[0,0] - 2 * torch.pi and q[0,1] <= q[0,0] - torch.pi):
        clockwise = False
    else:
        clockwise = True
    return clockwise


def wrap_to_pi(tensor):
    return (tensor + torch.pi) % (2 * torch.pi) - torch.pi

In [ ]:
Kp = torch.tensor([[100., 0.], [0., 0.]]).to(device)
Kd = torch.tensor([[40., 0.], [0., 0.]]).to(device)
Kp_old = 50.
Kd_old = 30.

In [ ]:
q_start = torch.tensor([[-torch.pi/2, -torch.pi/2+0.1]]).requires_grad_().to(device)
#q_start = torch.tensor([[-1.5244, -3.0952]]).requires_grad_().to(device)
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
#q_d_start = torch.tensor([[0.5, 0.5]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)

is_clockwise_start = check_clockwise(q_start)

xy_des_real = torch.tensor([0., 0.5]).requires_grad_().to(device)
q_des = transforms.inverse_kinematics(rp, xy_des_real, is_clockwise=False).unsqueeze(0)
#q_des = torch.tensor([[-torch.pi/2, -torch.pi/4]]).requires_grad_().to(device)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

th_start = model.encoder(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T
th_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Obtain accelerations in thetas

is_clockwise_des = check_clockwise(q_des)

th_des = model.encoder(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T

q_des_est = model.decoder(th_des, is_clockwise_des)
q_d_des_est = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
xy_des_est, _ = transforms.forward_kinematics(rp, q_des_est[0])

In [ ]:
th_series = th_start.clone().detach()
th_d_series = th_d_start.clone().detach()
th_dd_series = th_dd_start.clone().detach()


q_est_series = model.decoder(th_start, clockwise=is_clockwise_start)
q_d_est_series = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
q_dd_est_series = torch.zeros((1,2)).to(device)


q_real_series = q_start
q_d_real_series = q_d_start
q_dd_real_series = q_dd_start


dt = 0.01
t_end = 15

th = th_start
th_d = th_d_start
th_dd = th_dd_start

q_est = model.decoder(th_start, clockwise=is_clockwise_start)
q_d_est = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
q_dd_est = torch.zeros((1,2))

q_real = q_start
q_d_real = q_d_start
q_dd_real = q_dd_start

is_clockwise = check_clockwise(q_start)

feedforward = True
gravity = False

for t in torch.arange(0, t_end, dt):
    print("Time:", t.item().__round__(3))

    if feedforward:
        q = model.decoder(th, is_clockwise)
        q_d = (model.jacobian_dec(th, is_clockwise) @ th_d.T).T
    else:
        q = q_real
        q_d = q_d_real

        th = model.encoder(q)
        th_d = (model.jacobian_enc(q) @ q_d.T).T

    J_h = model.jacobian_enc(q)
    J_h_trans = torch.transpose(J_h, 0, 1)

    is_clockwise = check_clockwise(q)
    
    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q, q_d)
    A_q = dynamics.input_matrix(rp , q)


    """ Feed-forward simulation of the system, not on real dynamics """
    A_th = transforms.transform_input_matrix_from_inverse_trans(A_q, J_h_inv_trans, device)
    
    u = torch.pinverse(A_th) @ ((Kp @ (th_des - th).T + Kd @ (th_d_des - th_d).T))
    
    if feedforward:
        M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q, C_q, G_q, th, th_d, J_h_inv, J_h_inv_trans)

        if gravity:
            u = u + torch.pinverse(A_th) @ G_th
        tau_th = A_th * u

        print("u at th:", u)

        if gravity:
            th_dd = (torch.pinverse(M_th) @ (tau_th - C_th @ th_d.T - G_th)).T 
        else:
            th_dd = (torch.pinverse(M_th) @ (tau_th - C_th @ th_d.T)).T

        th = th.detach().requires_grad_()
        th_d = th_d.detach().requires_grad_()
        th_dd = th_dd.detach().requires_grad_()

        th_d = th_d + th_dd * dt
        th = th + th_d * dt

        q = q.detach().requires_grad_()
        q_d = q_d.detach().requires_grad_()
        #q_dd = q_dd.detach().requires_grad_()
        
        J_h_inv_new = model.jacobian_dec(th, is_clockwise).squeeze(0)

        q_d_old = q_d
        q_d = (J_h_inv_new @ th_d.T).T
        q_dd = (q_d - q_d_old)/dt
        q = model.decoder(th, is_clockwise)
        q = wrap_to_pi(q)
    

    """ Update the real system and apply latent control input. """

    M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_real, q_d_real)
    A_q_real = dynamics.input_matrix(rp, q_real)

    if gravity and not feedforward:
        u = u + torch.pinverse(A_q) @ G_q_real

    tau_q_real = A_q_real * u

    if gravity:
        q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T) - G_q_real)).T 
    else:
        q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T))).T
    
    q_d_real = q_d_real + q_dd_real * dt
    q_real = q_real + q_d_real * dt
    q_real = wrap_to_pi(q_real)
    
    if feedforward:
        q_est = q
        q_d_est = q_d
        q_dd_est = q_dd
    else:
        th_est = model.encoder(q_real)
        q_est = model.decoder(th_est, is_clockwise)
        th_d_est = (model.jacobian_enc(q_real) @ q_d_real.T).T
        q_d_est = (model.jacobian_dec(th_est, clockwise=is_clockwise) @ th_d_est.T).T
        q_dd_est = q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Implement with Jacobian derivative


    """ Store data for plotting """

    th_series = torch.cat((th_series, th.detach()), dim=0)
    th_d_series = torch.cat((th_d_series, th_d.detach()), dim=0)
    th_dd_series = torch.cat((th_dd_series, th_dd.detach()), dim=0)
    
    q_real_series = torch.cat((q_real_series, q_real.detach()), dim=0)
    q_d_real_series = torch.cat((q_d_real_series, q_d_real.detach()), dim=0)
    q_dd_real_series = torch.cat((q_dd_real_series, q_dd_real.detach()), dim=0)

    q_est_series = torch.cat((q_est_series, q_est.detach()), dim=0)
    q_d_est_series = torch.cat((q_d_est_series, q_d_est.detach()), dim=0)
    q_dd_est_series = torch.cat((q_dd_est_series, q_dd_est.detach()), dim=0) 

    #print(is_clockwise)
    print("")

    
    
#print(q_real_series)
#print(q_d_real_series)
#print(q_dd_real_series)
#print(q_est_series)
#print(q_d_est_series)
#print(q_dd_est_series)


In [ ]:
pos_end_q_real, pos_elbow_q_real = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_real_series)
pos_end_q_est, pos_elbow_q_est = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_est_series)
colors_q_real = cm.rainbow(np.linspace(0, 1, pos_end_q_real.size(0)))
colors_q_est = cm.rainbow(np.linspace(0, 1, pos_end_q_est.size(0)))
frames_q_real = plotter.frame_pendulum(pos_end_q_real, pos_elbow_q_real, colors = colors_q_real, height=5, width=5)
frames_q_est = plotter.frame_pendulum(pos_end_q_est, pos_elbow_q_est, colors = colors_q_est, height=5, width=5)

frames_data = [(frames_q_est, "q_est", "tab:orange", "tab:red"), (frames_q_real, "q_real", "tab:blue", "tab:cyan")]
#frames_data = [(frames_q_real, "q_real", "tab:blue", "tab:cyan")]
#frames_data = [(frames_q_est, "q_est", "tab:orange", "tab:red")]
file_name = "yahooooo" 
file_name_gif = file_name + ".gif"
file_name_mp4 = file_name + ".mp4"



plotter.animate_pendulum(frames_data, ref_pos=xy_des, actuator=True, file_name=file_name_mp4, fps = 100)
#plotter.animate_pendulum(frames_data, ref_pos=None, actuator=False, file_name=file_name_mp4, fps = 100)
print(len(frames_data[0][0]))

In [ ]:



q = torch.tensor([[torch.pi/4, 0.0]])
q1_in = q[0,0]
q2_in = q[0,1]

q_in_list = []
q_out_cw_list = []
q_out_ccw_list = []

q_in_tensors = torch.empty((0,2))



def theta_from_q(q1_in, q2_in):

    theta1 = torch.sqrt((rp["xa"] - rp["l1"]*torch.cos(q1_in) - rp["l2"]*torch.cos(q2_in))**2 + 
                        (rp["ya"] - rp["l1"]*torch.sin(q1_in) - rp["l2"]*torch.sin(q2_in))**2)

    theta2 = torch.atan2((rp["ya"] - rp["l1"]*torch.sin(q1_in) - rp["l2"]*torch.sin(q2_in)),
                        (rp["xa"] - rp["l1"]*torch.cos(q1_in) - rp["l2"]*torch.cos(q2_in)))
    #print("theta1:", theta1, "   theta2:", theta2)
    return(theta1, theta2)

def q_from_theta(theta1, theta2):
    xend = rp["xa"] - theta1*torch.cos(theta2)
    yend = rp["ya"] - theta1*torch.sin(theta2)


    numerator = (xend**2 + yend**2 - rp["l1"]**2 - rp["l2"]**2)
    denominator = torch.tensor(2*rp["l1"]*rp["l2"])

    beta = torch.arccos(numerator/(denominator+epsilon))
    q1 = torch.atan2(yend, xend + epsilon) - torch.atan2(rp["l2"]*torch.sin(beta), epsilon + rp["l1"] + rp["l2"]*torch.cos(beta))
    q2 = q1 + beta

    q1_alt = torch.atan2(yend, xend) + torch.atan2(rp["l2"]*torch.sin(beta), rp["l1"] + rp["l2"]*torch.cos(beta))
    q2_alt = q1_alt - beta 

    return ((q1, q2), (q1_alt, q2_alt))

def plot_coordinates(q_in_tensors, q_cw, q_ccw):
    """
    Plots three lists of (x, y) coordinates in separate subplots, side by side, with the same limits.

    Parameters:
    - q_in_list: List of (x, y) tuples to be plotted in the first subplot.
    - q_out_list: List of (x, y) tuples to be plotted in the second subplot.
    - q_alt_out_list: List of (x, y) tuples to be plotted in the third subplot.
    """
    # Unpack each list into x and y coordinates
    #x_in, y_in = zip(*q_in_list) if q_in_list else ([], [])
    #x_out, y_out = zip(*q_out_list) if q_out_list else ([], [])
    #x_alt_out, y_alt_out = zip(*q_alt_out_list) if q_alt_out_list else ([], [])

    x_in = q_in_tensors[:, 0].numpy()
    y_in = q_in_tensors[:, 1].numpy()
    x_out = q_cw[:, 0].numpy()
    y_out = q_cw[:, 1].numpy()
    x_alt_out = q_ccw[:, 0].numpy()
    y_alt_out = q_ccw[:, 1].numpy()

    colors = cm.rainbow(np.linspace(0, 1, len(x_in)))

    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

    # Define axis limits
    x_limits = (-1.1*np.pi, 1.1*np.pi)
    y_limits = (-1.1*np.pi, 1.1*np.pi)

    # Plot q_in_list
    axes[0].scatter(x_in, y_in, color=colors, label='q_in_list', alpha=0.7)
    axes[0].set_xlim(x_limits)
    axes[0].set_ylim(y_limits)
    axes[0].set_title('q_in_list')
    axes[0].set_xlabel('q1')
    axes[0].set_ylabel('q2')
    axes[0].grid(True)

    # Plot q_out_list
    axes[1].scatter(x_out, y_out, color=colors, label='q_out_list', alpha=0.7)
    axes[1].set_xlim(x_limits)
    axes[1].set_ylim(y_limits)
    axes[1].set_title('q_alt_out_list')
    axes[1].set_xlabel('X Coordinate')
    axes[1].grid(True)

    # Plot q_alt_out_list
    axes[2].scatter(x_alt_out, y_alt_out, color=colors, label='q_alt_out_list', alpha=0.7)
    axes[2].set_xlim(x_limits)
    axes[2].set_ylim(y_limits)
    axes[2].set_title('q_alt_out_list')
    axes[2].set_xlabel('q1')
    axes[2].grid(True)

    # Adjust layout
    plt.tight_layout()
    plt.show()

epsilon = 0.0001

for j in range(0,1):
    for i in range(0,120):
        q1_in = torch.tensor(i*torch.pi/60 + j*torch.pi/40)
        q2_in = torch.tensor(j*torch.pi/5) - q1_in
        q_in_tensors = torch.cat((q_in_tensors, torch.tensor([[q1_in, q2_in]])), dim=0)

th = torch.vmap(transforms.analytic_theta, in_dims=(None, 0))(rp, q_in_tensors)
q_cw, q_ccw = torch.vmap(transforms.analytic_inverse, in_dims=(None, 0))(rp, th)

plot_coordinates(q_in_tensors, q_cw, q_ccw)




#plot_coordinates(q_in_list, q_out_cw_list, q_out_ccw_list)



for q_in, q_out_cw, q_out_ccw in zip(q_in_tensors, q_cw, q_ccw):

    pos_end, pos_elbow = transforms.forward_kinematics(rp, q_in)
    

    #plotter.plot_double_pendulum([pos_end], [pos_elbow])


pos_end_tensor, pos_elbow_tensor = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_in_tensors)
pos_end_tensor_cw, pos_elbow_tensor_cw = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_cw)
pos_end_tensor_ccw, pos_elbow_tensor_ccw = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_ccw)
colors = cm.rainbow(np.linspace(0, 1, pos_end_tensor.size(0)))
frames_input = plotter.frame_pendulum(pos_end_tensor, pos_elbow_tensor, colors = colors, height=5, width=5)
frames_cw = plotter.frame_pendulum(pos_end_tensor_cw, pos_elbow_tensor_cw, colors = colors, height=5, width=5)
frames_ccw = plotter.frame_pendulum(pos_end_tensor_ccw, pos_elbow_tensor_ccw, colors = colors, height=5, width=5)

print(pos_end_tensor)
print(pos_elbow_tensor)



In [ ]:
# Set up q_est and q_real to be the same
q_est = torch.tensor([[-torch.pi/4, -torch.pi/2]]).requires_grad_(True)
q_d_est = torch.tensor([[torch.pi+0.1, -torch.pi/2]]).requires_grad_(True)

# Calculate th
th = model.encoder(q_est)
th_d = (model.jacobian_enc(q_est) @ q_d_est.T).T

# Calculate Jacobians
J_h = model.jacobian_enc(q_est)
J_h_trans = torch.transpose(J_h, 0, 1)

J_h_inv = model.jacobian_dec(th)
J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

J_h = model.jacobian_enc(q_est)
J_h_trans = torch.transpose(J_h, 0, 1)

J_h_inv = model.jacobian_dec(th)
J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

# Obtain dynamic matrices in q-space
M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_est.squeeze(0), q_d_est.squeeze(0))
A_q_est = dynamics.input_matrix(rp , q_est.squeeze(0))

# Obtain dynamic matrices in theta-space
M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans, device)

# Reconstruct dynamical matrices from theta-space to q-space
M_recon = J_h_trans @ M_th @ J_h
C_recon = torch.zeros(M_recon.size()).to(M_recon.device)

for i in range(C_recon.size(0)):
    for j in range(C_recon.size(1)):
        for k in range(C_recon.size(1)):
            M_recon_dot_ijk = torch.autograd.grad(M_recon[i,j], q_est, create_graph=True)[0][0,k]
            M_recon_dot_ikj = torch.autograd.grad(M_recon[i,k], q_est, create_graph=True)[0][0,j]
            M_recon_dot_jki = torch.autograd.grad(M_recon[j,k], q_est, create_graph=True)[0][0,i]

            C_recon[i, j] += 0.5 * (M_recon_dot_ijk + M_recon_dot_ikj - M_recon_dot_jki) * q_d_est[0, k]

G_recon = J_h_trans @ G_th


print("Mqest:\n", M_q_est)  
print("Mrecon:\n", M_recon)

print("Cqest:\n", C_q_est)
print("Crecon:\n", C_recon)

print("Gqest:\n", G_q_est)
print("Grecon:\n", G_recon)

1. Give setpoint in q-space
2. Give current position in q
3. Map both to theta
4. Calculate distance 